## Overview  

In this project I am working with data from Zillow to try to predict sales prices in the future. The goal is to see which zipcodes would make the most sense to invest in with regards to profit and return on investment.  

I will look at all three of these parameters when making my recommendations for these reasons: 
  
**Profit** - Tells us in dollar amount how much we estimate can be made as well as confidence intervels with a min and max profit margin.  
  
**Return on investment** - Percentage that can be gained, this metric can tell us if the amount we are able to make is a small or big precentage. This will help with understanding that even if a big profit can be made, this may be more of a risk with regards to small market swings. 

  
### City  

I will be looking at zipcodes in the city of Chicago, of which there are 41 zipcodes. 


# Mod 4 Project - Starter Notebook

This notebook has been provided to you so that you can make use of the following starter code to help with the trickier parts of preprocessing the Zillow dataset. 

The notebook contains a rough outline the general order you'll likely want to take in this project. You'll notice that most of the areas are left blank. This is so that it's more obvious exactly when you should make use of the starter code provided for preprocessing. 

**_NOTE:_** The number of empty cells are not meant to infer how much or how little code should be involved in any given step--we've just provided a few for your convenience. Add, delete, and change things around in this notebook as needed!

# Some Notes Before Starting

This project will be one of the more challenging projects you complete in this program. This is because working with Time Series data is a bit different than working with regular datasets. In order to make this a bit less frustrating and help you understand what you need to do (and when you need to do it), we'll quickly review the dataset formats that you'll encounter in this project. 

## Wide Format vs Long Format

If you take a look at the format of the data in `zillow_data.csv`, you'll notice that the actual Time Series values are stored as separate columns. Here's a sample: 

<img src='~/../images/df_head.png'>

You'll notice that the first seven columns look like any other dataset you're used to working with. However, column 8 refers to the median housing sales values for April 1996, column 9 for May 1996, and so on. This This is called **_Wide Format_**, and it makes the dataframe intuitive and easy to read. However, there are problems with this format when it comes to actually learning from the data, because the data only makes sense if you know the name of the column that the data can be found it. Since column names are metadata, our algorithms will miss out on what dates each value is for. This means that before we pass this data to our ARIMA model, we'll need to reshape our dataset to **_Long Format_**. Reshaped into long format, the dataframe above would now look like:

<img src='~/../images/melted1.png'>

There are now many more rows in this dataset--one for each unique time and zipcode combination in the data! Once our dataset is in this format, we'll be able to train an ARIMA model on it. The method used to convert from Wide to Long is `pd.melt()`, and it is common to refer to our dataset as 'melted' after the transition to denote that it is in long format. 

# Helper Functions Provided

Melting a dataset can be tricky if you've never done it before, so you'll see that we have provided a sample function, `melt_data()`, to help you with this step below. Also provided is:

* `get_datetimes()`, a function to deal with converting the column values for datetimes as a pandas series of datetime objects
* Some good parameters for matplotlib to help make your visualizations more readable. 

Good luck!


# Step 1: Load the Data/Filtering for Chosen Zipcodes

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
from matplotlib.pylab import rcParams
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
import itertools
from sklearn.metrics import mean_squared_error
from tqdm import tqdm

In [ ]:
df = pd.read_csv('zillow_data.csv')

In [ ]:
df.head()

In [ ]:
df1 = df.loc[df['City'] == 'Chicago']

In [ ]:
df1.head()

In [ ]:
len(df1)

Checking for Null values

In [ ]:
df.isna().sum()

Checking to see where those null values are

In [ ]:
df1

Seeing where this zipcode actually starts, as I will need to filter for that in the modeling later

In [ ]:
df60611 =  df1.loc[df1['RegionName'] == 60611]

In [ ]:
df60611.iloc[:, 214:]

# Step 2: Reshape from Wide to Long Format

In [ ]:
def melt_data(df):
    melted = pd.melt(df, id_vars=['RegionID', 'RegionName', 'City', 'State', 'Metro', 'CountyName', 'SizeRank'], var_name='time')
    melted['time'] = pd.to_datetime(melted['time'], infer_datetime_format=True)
    melted = melted.dropna(subset=['value'])
    return melted.groupby('time').aggregate({'value':'mean'})

In [ ]:
def melt_df(df):
    merged = []
    for zipcode in df.RegionName:
        melted = melt_data(df.loc[df['RegionName'] == zipcode])
        row = df.loc[df['RegionName'] == zipcode].iloc[:,:7]
        rows = pd.concat([row]*len(melted), ignore_index=True)
        merge = pd.concat([rows, melted.reset_index()], axis= 1)
        merged.append(merge)
    melted_df = pd.concat(merged)
    return melted_df

In [ ]:
melted_df1 = melt_df(df1)

In [ ]:
melted_df1.isna().sum()

In [ ]:
melted_df1.head()

In [ ]:
len(melted_df1)

Set the time column as the index

In [ ]:
melted_df1.set_index('time', inplace=True)

# Step 3: EDA and Visualization

In [ ]:
font = {'family' : 'normal',
        'weight' : 'bold',
        'size'   : 12}

plt.rc('font', **font)

# NOTE: if your visualizations are too cluttered to read, try calling 'plt.gcf().autofmt_xdate()'!

Making a general visual, just to see the general trend of all the zipcodes

In [ ]:
plt.figure(figsize=(12,8))
for zipcode in melted_df1.RegionName.unique():
    melted_df1.loc[melted_df1['RegionName'] == zipcode].value.plot(label=zipcode)
plt.legend()
plt.show()
    

I want to checkout the upper part of this graph as the lower part seems to not have much change from the start to the end. As it seems that the ones that have the most change start higher as well, I will just filter for the 10 best means and that should give me what I am looking for.

In [ ]:
zips = pd.DataFrame()
zipsdict = {}
for zipcode in melted_df1.RegionName.unique():
    zipsdict[zipcode] = melted_df1.loc[melted_df1['RegionName'] == zipcode].value.mean()
zips['zipcode'] = zipsdict.keys()
zips['means'] = zipsdict.values()
highmean_list = list(zips.sort_values(by='means', ascending=False).zipcode[:10])

In [ ]:
plt.figure(figsize=(12,8))
for zipcode in highmean_list:
    zipdf = melted_df1.loc[melted_df1['RegionName'] == zipcode].value
    zipdf.plot(label=zipcode)
plt.legend()
plt.show()

Seems I got what I am looking for, from 1996 until 2019, these seem to be the most growing zipcodes, though what effect that will have on future growth remains to be seen. Might be interesting to check back at this after I figure out the best ones by predictionto see if they match up or not

Just want to see a basic breakdown of one zipcode to see the seasonality and trends.

In [ ]:
decomposition = seasonal_decompose(melted_df1.loc[melted_df1['RegionName'] == 60654].value)
decomposition.plot();

Seems to be very seasonal and the residuals seem to be more volatile from the start of the corresponding drop in the main graph, which seems to relate to the market crash.

# Step 5: ARIMA Modeling

Making a test on one zipcode to then use the code for everything

In [ ]:
test = melted_df1.loc[melted_df1['RegionName'] == 60654]

In [ ]:
ARIMA_MODEL = sm.tsa.statespace.SARIMAX(test.value,
                                order=(0, 1, 1),
                                seasonal_order=(0, 1, 1, 12), 
                                enforce_stationarity=False, 
                                enforce_invertibility=False)

# Fit the model and print results
output = ARIMA_MODEL.fit()
display(output.summary().tables[0], output.summary().tables[1], output.summary().tables[2])

Printing the diagnostics of the model

In [ ]:
output.plot_diagnostics(figsize=(15, 18))
plt.show()

Checking getting the predictions along with the lower and upper confidence intervals

In [ ]:
pred = output.get_prediction(start=pd.to_datetime('2015-01-01'), dynamic=False)
pred_conf = pred.conf_int()

In [ ]:
b = {}
b['a'] = pred_conf[pred_conf.index == '2015-02-01']['lower value'][0]

In [ ]:
pred_conf[pred_conf.index == '2015-02-01']['lower value']

In [ ]:
b

In [ ]:
pred_conf['upper value'].mean()

In [ ]:
pred.predicted_mean['2018-01-01']

Making a graph of the predictions and the original data

In [ ]:
rcParams['figure.figsize'] = 15, 6

#Plot observed values
ax = test['2014':].value.plot(label='observed')

#Plot predicted values
pred.predicted_mean.plot(ax=ax, label='Forecast', alpha=.7)

#Plot the range for confidence intervals
ax.fill_between(pred_conf.index,
                pred_conf.iloc[:, 0],
                pred_conf.iloc[:, 1], color='y', alpha=.5)

#Set axes labels
ax.set_xlabel('Date')
ax.set_ylabel('Values')
plt.legend()

plt.show()

Checking the RMSE

In [ ]:
test_forecasted = pred.predicted_mean
test_truth = test['2017':].value
test_forecasted = test_forecasted['2017':]
# Compute the root mean square error
error = mean_squared_error(test_forecasted, test_truth)
error = np.sqrt(error)
print(f'The RMSE of our forecasts is {error}')

Our predictions are within about 380,000 dollars. Meaning about give or take 140,000 dollars. This is not too bad concidering that this zipcode has houses over a million dollars

I would like to figure out the best parameters to use for the model, based on AIC.  
First I will generate the possible combinations and then get the best one for each zipcode

In [ ]:
# Define the p, d and q parameters to take any value between 0 and 2
p = d = q = range(0, 2)

# Generate all different combinations of p, q and q triplets
pdq = list(itertools.product(p, d, q))

# Generate all different combinations of seasonal p, q and q triplets
pdqs = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))]

In [ ]:
best_combs = []
for zipcode in tqdm(melted_df1.RegionName.unique()):
    ans = []
    for comb in pdq:
        for combs in pdqs:
            try:
                mod = sm.tsa.statespace.SARIMAX(melted_df1.loc[melted_df1['RegionName'] == zipcode].value,
                                                order=comb,
                                                seasonal_order=combs,
                                                enforce_stationarity=False,
                                                enforce_invertibility=False)

                output = mod.fit()
                ans.append([zipcode, comb, combs, output.aic])
    #             print(f'ARIMA {comb} x {combs}12 : AIC Calculated ={output.aic}')
            except:
                continue
    best_combs.append(sorted(ans, key=lambda x: x[3])[0])

In [ ]:
best_combs

These are the best combinations for each zipcode, using 12 for seasonality and a 0 or a 1 for everything else.  
I will now try to check changing everything, including seasonality to see which ones are the best

In [ ]:
# Define the p, d and q parameters to take any value between 0 and 10
p = d = q = range(0, 11)

# Generate all different combinations of p, q and q triplets
pdq = list(itertools.product(p, d, q))

# Define the p, d, q and s parameters to take any value between 0 and 10
p2 = d2 = q2 = s = range(0, 11)

# Generate all different combinations of seasonal p, q and q triplets
pdqs = [(x[0], x[1], x[2], x[3]) for x in list(itertools.product(p2, d2, q2, s))]

In [ ]:
len(pdqs)

Make a function for creating an image with the original data, and the predicted data, along with the forcasted data and the confidence intervals

In [ ]:
def make_pred_image(df, test_forecasted, forcast, zipcode=None):
    rcParams['figure.figsize'] = 15, 6

    #Plot observed values
    ax = df['2015':].plot(label='observed')

    #Plot predicted values
    test_forecasted['2015-01-01':].plot(ax=ax, label='Predicted', alpha=.7)
    
    forcast.predicted_mean.plot(ax=ax, label='Forecast', alpha=.7)

    #Plot the range for confidence intervals
    ax.fill_between(pred_conf.index,
                    pred_conf.iloc[:, 0],
                    pred_conf.iloc[:, 1], color='y', alpha=.5)

    #Set axes labels
#     ax.title(zipcode)
    ax.set_xlabel('Date')
    ax.set_ylabel('Values')
    plt.legend()

    plt.show()
    return None

I am using a loop to get predictions for each zipcode based on the best combinations of parameters.  
It will also print out a graph for each zipcode along with the diagnostics.  
I put all the metrics I want into dictionaries with the zipcode as the key and the metric as the value.  
I also setup a dataframe and then make each dictionary correspond to a column in the dataframe

In [ ]:
pred_df = pd.DataFrame()
known_RMSE = {}
forcast_2019_04 = {}
confid_min = {}
confid_max = {}
profit = {}
roi = {}
for comb in best_combs:
    
    if comb[0] == 60611:
        
        df = melted_df1.loc[melted_df1['RegionName'] == comb[0]].value.dropna()
        
        mod = sm.tsa.statespace.SARIMAX(df,
                                    order=comb[1],
                                    seasonal_order=comb[2],
                                    enforce_stationarity=False,
                                    enforce_invertibility=False,)

        output = mod.fit()
        pred = output.get_prediction(start=pd.to_datetime('2013-07-01'), dynamic=False)
        test_forecasted = pred.predicted_mean
        test_truth = test['2013-07-01':].value
        test_forecasted = test_forecasted['2013-07-01':]
    else:
        
        df = melted_df1.loc[melted_df1['RegionName'] == comb[0]].value.dropna()
       
        mod = sm.tsa.statespace.SARIMAX(df,
                                    order=comb[1],
                                    seasonal_order=comb[2],
                                    enforce_stationarity=False,
                                    enforce_invertibility=False)

        output = mod.fit()
        pred = output.get_prediction(start=pd.to_datetime('1996-04-01'), dynamic=False)
        test_forecasted = pred.predicted_mean
        test_truth = test['1996-04-01':].value
        test_forecasted = test_forecasted['1996-04-01':]
    error = mean_squared_error(test_forecasted, test_truth)
    known_RMSE[comb[0]] = np.sqrt(error)
    forcast = output.get_forecast(steps=24)
    forcast_2019_04[comb[0]] = forcast.predicted_mean['2019-04-01']
    pred_conf = forcast.conf_int()
    confid_min[comb[0]] = pred_conf[pred_conf.index == '2019-04-01']['lower value'][0]
    confid_max[comb[0]] = pred_conf[pred_conf.index == '2019-04-01']['upper value'][0]
    val_04_18 = melted_df1.loc[melted_df1['RegionName'] == comb[0]].value[-1]
    profit[comb[0]] = forcast.predicted_mean['2019-04'] - val_04_18
    roi[comb[0]] = profit[comb[0]]/val_04_18
    
    print(comb[0])
    make_pred_image(df, test_forecasted, forcast)
    output.plot_diagnostics(figsize=(12, 15))
    plt.show()
    
    

pred_df['zipcode'] = known_RMSE.keys()
pred_df['known_RMSE'] = known_RMSE.values()
pred_df['forcast_2019_04'] = forcast_2019_04.values()
pred_df['confid_min'] = confid_min.values()
pred_df['confid_max'] = confid_max.values()
pred_df['profit'] = profit.values()
pred_df['roi'] = roi.values()

Cleaning up how the data is displayed to make it easier to read in the dataframe

In [ ]:
pred_df['roi'] = pred_df['roi'].apply(lambda x: round(x[0], 2))

In [ ]:
pred_df['profit'] = pred_df['profit'].apply(lambda x: round(x[0]))

In [ ]:
pred_df['2018_04_price'] = pred_df['forcast_2019_04'] - pred_df['profit']

In [ ]:
pred_df['2018_04_price'] = pred_df['2018_04_price'].apply(lambda x: round(x))

In [ ]:
pred_df['forcast_2019_04'] = pred_df['forcast_2019_04'].apply(lambda x: round(x))

In [ ]:
pred_df['known_RMSE'] = pred_df['known_RMSE'].apply(lambda x: round(x, 2))

In [ ]:
pred_df['confid_min'] = pred_df['confid_min'].apply(lambda x: round(x))

In [ ]:
pred_df['confid_max'] = pred_df['confid_max'].apply(lambda x: round(x))

In [ ]:
pred_df.sort_values(by=['profit', 'roi'], ascending=False).head(10)

In [ ]:
pred_df_d = pd.DataFrame()
known_RMSE = {}
forcast_2019_04 = {}
confid_min = {}
confid_max = {}
profit = {}
roi = {}
for comb in best_combs:
    
    if comb[0] == 60611:
        
        df = melted_df1.loc[melted_df1['RegionName'] == comb[0]].value.dropna()
        
        mod = sm.tsa.statespace.SARIMAX(df,
                                    order=comb[1],
                                    seasonal_order=comb[2],
                                    enforce_stationarity=False,
                                    enforce_invertibility=False,)

        output = mod.fit()
        pred = output.get_prediction(start=pd.to_datetime('2013-07-01'), dynamic=True)
        test_forecasted = pred.predicted_mean
        test_truth = test['2013-07-01':].value
        test_forecasted = test_forecasted['2013-07-01':]
    else:
        
        df = melted_df1.loc[melted_df1['RegionName'] == comb[0]].value.dropna()
       
        mod = sm.tsa.statespace.SARIMAX(df,
                                    order=comb[1],
                                    seasonal_order=comb[2],
                                    enforce_stationarity=False,
                                    enforce_invertibility=False)

        output = mod.fit()
        pred = output.get_prediction(start=pd.to_datetime('1996-04-01'), dynamic=True, full_results=True)
        test_forecasted = pred.predicted_mean
        test_truth = test['1996-04-01':].value
        test_forecasted = test_forecasted['1996-04-01':]
    error = mean_squared_error(test_forecasted, test_truth)
    known_RMSE[comb[0]] = np.sqrt(error)
    forcast = output.get_forecast(steps=24, dynamic=True)
    forcast_2019_04[comb[0]] = forcast.predicted_mean['2019-04-01']
    pred_conf = forcast.conf_int()
    confid_min[comb[0]] = pred_conf[pred_conf.index == '2019-04-01']['lower value'][0]
    confid_max[comb[0]] = pred_conf[pred_conf.index == '2019-04-01']['upper value'][0]
    val_04_18 = melted_df1.loc[melted_df1['RegionName'] == comb[0]].value[-1]
    profit[comb[0]] = forcast.predicted_mean['2019-04'] - val_04_18
    roi[comb[0]] = profit[comb[0]]/val_04_18
    
    print(comb[0])
    make_pred_image(df, test_forecasted, forcast)
    output.plot_diagnostics(figsize=(12, 15))
    plt.show()
    
    

pred_df_d['zipcode'] = known_RMSE.keys()
pred_df_d['known_RMSE'] = known_RMSE.values()
pred_df_d['forcast_2019_04'] = forcast_2019_04.values()
pred_df_d['confid_min'] = confid_min.values()
pred_df_d['confid_max'] = confid_max.values()
pred_df_d['profit'] = profit.values()
pred_df_d['roi'] = roi.values()

In [ ]:
pred_df_d['roi'] = pred_df_d['roi'].apply(lambda x: round(x[0], 2))

In [ ]:
pred_df_d['profit'] = pred_df_d['profit'].apply(lambda x: round(x[0]))

In [ ]:
pred_df_d['2018_04_price'] = pred_df_d['forcast_2019_04'] - pred_df['profit']

In [ ]:
pred_df_d['2018_04_price'] = pred_df_d['2018_04_price'].apply(lambda x: round(x))

In [ ]:
pred_df_d['forcast_2019_04'] = pred_df_d['forcast_2019_04'].apply(lambda x: round(x))

In [ ]:
pred_df_d['known_RMSE'] = pred_df_d['known_RMSE'].apply(lambda x: round(x, 2))

In [ ]:
pred_df_d['confid_min'] = pred_df_d['confid_min'].apply(lambda x: round(x))

In [ ]:
pred_df_d['confid_max'] = pred_df_d['confid_max'].apply(lambda x: round(x))

In [ ]:
pred_df_d.sort_values(by=['profit', 'roi'], ascending=False).head(10)

In [ ]:
pred_df.sort_values(by=['profit', 'roi'], ascending=False).head(10)

# Step 6: Interpreting Results